In [ ]:
# for high bandwidth model downloading
!pip install hf_transfer
import os
os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = '1'

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import EvalPrediction

from datasets import load_dataset, Split

from trl import SFTTrainer

from typing import Dict

# target_model_id = 'NousResearch/Llama-2-7b-hf'
# supervisor_model_id = 'NousResearch/Llama-2-7b-hf'

target_model_id = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'
supervisor_model_id = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'

In [ ]:
# Download and load models
target_tokenizer = AutoTokenizer.from_pretrained(target_model_id)
target_model = AutoModelForCausalLM.from_pretrained(target_model_id)

supervisor_tokenizer = AutoTokenizer.from_pretrained(supervisor_model_id)
supervisor_model = AutoModelForCausalLM.from_pretrained(supervisor_model_id)

In [ ]:
# Load TruthfulQA dataset. VALIDATION is the only available split
all_data = load_dataset(path='truthful_qa', name='generation', split=Split.VALIDATION).train_test_split(train_size=0.66, shuffle=True, seed=42)

In [ ]:
def compute_metrics(eval_pred: EvalPrediction) -> Dict[str, float]:
    print('COMPUTING METRICS')
    predictions, labels = eval_pred
    # TODO: measure alignment using `supervisor_model`
    return {}

In [ ]:
trainer = SFTTrainer(
    model=target_model,
    train_dataset=all_data[Split.TRAIN],
    eval_dataset=all_data[Split.TEST],
    compute_metrics=compute_metrics,
    max_seq_length=1024,
    dataset_text_field='question',
    dataset_batch_size=32
)

In [ ]:
trainer.train()